```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
from tools.core.utils import FileManager

i = 2
file_manager = FileManager()
excel = file_manager.load_excel_metadata(i)

In [ ]:
excel.columns

In [ ]:
import shutil
from tqdm import tqdm
import os, json

# zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
zip_path = '/Users/jinym/Downloads/14.251124_2,3차잔여도서_원천코퍼스'
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"
final_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/{i}C"

error_list = []
for id in tqdm(os.listdir(zip_path)):
# if True:
   # id = 'SS0174'
  if not id.startswith('SS'):
    continue

  id = id.replace('.zip', '')
   
  try:
      lv = excel.loc[id, '분류']
  except Exception as e:
      error_list.append({'id':id, 'error': "excel error"})
      continue
  
  if '/' in lv: 
      lv = lv.replace("/", '_')

  if 'Lv' not in lv:
      continue
  
  # os.system(f"cp {zip_path}/{id}.zip {original_path}/{lv}/{id}.zip && cd {original_path}/{lv} && mkdir {id} && unzip {id}.zip -d {id}")
  # os.system(f"mkdir {final_path}/{lv}/{id} && cp {original_path}/{lv}/{id}/{id}.json {final_path}/{lv}/{id}/ && cp -r {original_path}/{lv}/{id}/crop {final_path}/{lv}/{id}/")
                # && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")
  # 위에 실패했을 때 따로
  os.system(f"cp {original_path}/{lv}/{id}/{id}.json {final_path}/{lv}/{id}/ && cp -r {original_path}/{lv}/{id}/crop {final_path}/{lv}/{id}/")

  try:
      file_path = f"{final_path}/{lv}/{id}/{id}.json"
      with open(file_path, 'r', encoding='utf-8') as f:
          content = json.load(f)
        
      with open(file_path.replace(".json", ".json.bak"), 'w', encoding='utf-8') as f:
          json.dump(content, f, ensure_ascii=False, indent=4)

      updated_content = {
            'file_id': id,
            'ISBN': str(excel.loc[id, 'ISBN']),
            'title': excel.loc[id, '도서명'],
            'cat1_domain': excel.loc[id, '코퍼스 1분류'],
            'cat2_sub': excel.loc[id, '코퍼스 2분류'],
            'cat3_specific': excel.loc[id, '비고'],
            'pub_date': str(excel.loc[id, '출판일'])[:10],
            # 'contents': [],
            'contents': content['contents']
        }
      # break
      with open(file_path, 'w', encoding='utf-8') as f:
               json.dump(updated_content, f, ensure_ascii=False, indent=4)
  
  except Exception as e:
      error_list.append({'id':id, 'error': e})

In [ ]:
error_list

### 업로드

In [ ]:
import os

from_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/8_multiple_exam_+"
to_path = "yjmoon@oci004:/home/yjmoon/"
# to_path = "yjmoon@oci004:/srv4-data/vanilla_models/"


# # 업로드
os.system(f"scp -r {from_path} {to_path}")
# # for ss in file:
#     # os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
import os
os.system("scp -r yjmoon@oci004:/home/yjmoon/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/8_multiple_exam_+/exam_+_result/* /Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/8_multiple_exam_+/exam_+_result")

In [ ]:
# for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
#     fn = str(fn)
#     if '/' in lv: 
#        lv = lv.replace("/", '_')
# #     print(fn, tn, lv)
    
#     dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
#     os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
#     print(os.path.join(dir, fn))
#     # break

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

## 전체 돌리기

In [ ]:
import json
import tools.ProcessLv2 as pl2

path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv5'
for file in os.listdir(path):
    if not file.startswith('SS'):
        continue

    INPUT_PATH = os.path.join(path, file, file+'.json')
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    # with open(BACKUP_PATH, "w", encoding="utf-8") as f:
    #     json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # break
    id = os.path.splitext(os.path.basename(file))[0]
    try:
        new = {
            'file_id': id,
            'ISBN': str(excel.loc[id, 'ISBN']),
            'title': excel.loc[id, '도서명'],
            'cat1_domain': excel.loc[id, '코퍼스 1분류'],
            'cat2_sub': excel.loc[id, '코퍼스 2분류'],
            'cat3_specific': excel.loc[id, '비고'],
            'pub_date': str(excel.loc[id, '출판일'])[:10],
            # 'contents': [],
            'contents': origin['contents']
        }
    except:
        new = origin
    
    # break
    # new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    # new = pl2.erase_page(origin, 1)
    # new = pl2.format_change(3,origin)
    # new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
    # # with open(INPUT_PATH.replace(".json", "_new.json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)

## Lv2 pdf_page 만들기

In [ ]:
# import pandas as pd
# import os

# base_path = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
# analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
# buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
# i = 3

# excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3, engine='openpyxl')[['관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
# lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
# lv
# # print(lv == 'Lv2')

In [ ]:
# # Fixed version - using Python file operations instead of sed
# import shutil
# from tqdm import tqdm
# import fitz

# dpi = 300
# mat = fitz.Matrix(dpi / 150, dpi / 150)

# # for tn, isbn, lv in tqdm(zip(excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
# for tn in tqdm(excel_analy['관리번호']):
#    # if '/' in lv: 
#    #    lv = lv.replace("/", '_')
   
#    # if 'Lv' not in lv:
#    #    continue
#    lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
#    # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
#    if lv == 'Lv2':
#       os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2 && mkdir {tn} && mv {tn}*.json {tn}/ && cd {tn} && mkdir pdf_page")
#       # pdf_page 만들기
#       pdf_path = f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv1/{tn}.pdf"
#       doc = fitz.open(pdf_path)
#       for p, page in enumerate(doc):
#          img = page.get_pixmap(matrix=mat)
#          img.save(f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2/{tn}/pdf_page/{tn}_{p+1:04d}.png")
      
#       break

#    # SS0041 - SS0045 사이에서 MuPDF error: syntax error: invalid key in dict

### 1C Lv2/3/4 문제에서 부족한 문제 뽑기